# Градиентный бустинг

Бустинг &mdash; один из самых широко применяемых видов ансамблей моделей. В этом ноутбуке будет разобран самый простой вариант бустинга над деревьями, реализованный в `sklearn`. Для задач регрессии и классификации реализованы, соответственно, классы `sklearn.ensemble.GradientBoostingRegressor` и `sklearn.ensemble.GradientBoostingClassifier`.

## 1. Основные параметры градиентного бустинга.

* `learning rate` &mdash; размер шага метода оптимизации (стандартное значение=0.1);
* `n_estimators` &mdash; количество деревьев, над которыми будет выполняться бустинг (стандартное значение=100);
* `subsample` &mdash; доля выборки, на которой будут обучаться базовые модели, для каждой базовой модели &mdash; своя подвыборка (стандартное значение=1.0). Уменьшение этого параметра позволит сделать деревья менее переобученными, но более смещёнными;
* `min_samples_split` (стандартное значение=2);
* `min_samples_leaf` (стандартное значение=1);
* `max_depth` &mdash; ограничение на глубину деревьев (стандартное значение=3).

Про все возможные гиперпараметры вы можете прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html).

In [2]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

sns.set(font_scale=1.8, palette='Set2')

## 2. Подбор параметров градиентного бустинга

Будем решать градиентным бустингом задачу классификации. Возьмём [датасет](https://archive.ics.uci.edu/ml/datasets/Letter+Recognition) для распознавания латинских букв на изображениях.
     
Некоторые из признаков, содержащихся в датасете:
1.	`lettr` &mdash; заглавная буква	(принимает значения от A до Z);
2.	`x-box` &mdash; горизонтальная позиция прямоугольника с буквой;
3.	`y-box` &mdash; вертикальная позиция прямоугольника с буквой;
4.	`width`	&mdash; ширина прямоугольника;
5.	`high` &mdash; высота прямоугольника;
6.	`onpix` &mdash; количество пикселей, относящихся к цифре;
7. `x-bar` &mdash; среднее значение x всех пикселей в прямоугольнике;
8. `y-bar` &mdash; среднее значение y всех пикселей в прямоугольнике;
9. `x2-bar` &mdash; выборочная дисперсия x;
10. `y2-bar` &mdash; выборочная дисперсия y;
11. `xybar` &mdash; корреляция x и y.

In [3]:
letters_df = pd.read_csv('letter-recognition.data', header=None)
print('shape:', letters_df.shape)
letters_df.head()

shape: (20000, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


Деление на признаки и таргет:

In [4]:
X = letters_df.values[:, 1:]
y = letters_df.values[:, 0]

Разобьём данные на обучающую и тестовую выборки:

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### 2.1 Использование `GridSearchCV`

Для начала попробуем воспользоваться поиском по сетке с кросс-валидацией для поиска оптимальных гиперпараметров. Проблема в том, что если взять три параметра: `n_estimators`, `max_depth`, `learning_rate`, то перебор будет очень долгим. Поэтому ограничимся лишь `n_estimators`.

In [6]:
boosting_gridsearch = GridSearchCV(
    estimator=GradientBoostingClassifier(),
    param_grid={
        'n_estimators': [5, 10, 25, 50, 75], 
    },
    cv=5,  # разбиение выборки на 5 фолдов
    verbose=10,  # насколько часто печатать сообщения
    n_jobs=2  # кол-во параллельных процессов
)

Выполнение поиска по сетке. В процессе подбора гиперпараметров он иногда печатает информацию о том, сколько итераций он уже сделал, и сколько это заняло времени. Контролировать это можно с помощью параметра `verbose`.

In [7]:
boosting_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_jobs=2,
             param_grid={'n_estimators': [5, 10, 25, 50, 75]}, verbose=10)

Подобранные гиперпараметры:

In [8]:
boosting_gridsearch.best_params_

{'n_estimators': 75}

Смотрим на качество при оптимальных гиперпараметрах:

In [9]:
print('train accuracy {:.4f}'.format(
    accuracy_score(boosting_gridsearch.predict(X_train), y_train)
))
print('test accuracy {:.4f}'.format(
    accuracy_score(boosting_gridsearch.predict(X_test), y_test)
))

train accuracy 0.9427
test accuracy 0.9088


### 2.2 Использование `RandomizedSearchCV`

Во-первых, полный перебор по сетке может работать слишком долго. 
Во-вторых, при подборе более чем одного параметра **рекомендуется использовать случайный поиск**. Об этом можно почитать <a href="http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf">статью</a>.

Воспользуемся рандомизированным поиском по сетке, реализованным в `sklearn` как `RandomizedSearchCV`.

In [10]:
rnd_boosting_gridsearch = RandomizedSearchCV(
    GradientBoostingClassifier(),
    param_distributions={
        'n_estimators': [5, 10, 25, 50, 75],
        'learning_rate': np.linspace(0.05, 0.25, 5),
        'min_samples_leaf': np.arange(1, 6),
        'max_depth': [3, 4, 5, 6, None],
    },
    cv=5,  # разбиение выборки на 5 фолдов
    verbose=10,  # насколько часто печатать сообщения
    n_jobs=2,  # кол-во параллельных процессов
    n_iter=30,  # кол-во итераций случайного выбора гиперпараметров
    random_state=42  # seed для фиксирования случайности
)

Выполнение случайного поиска. На каждой из 30 итераций производится выбор случайных гиперпараметров, которые используются для проверки по каждому из 5 фолдов.

In [11]:
rnd_boosting_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV 2/5; 1/5] START n_estimators=5..............................................
[CV 2/5; 1/5] END ...............n_estimators=5;, score=0.696 total time=   1.6s
[CV 4/5; 1/5] START n_estimators=5..............................................
[CV 4/5; 1/5] END ...............n_estimators=5;, score=0.687 total time=   1.6s
[CV 1/5; 2/5] START n_estimators=10.............................................
[CV 1/5; 2/5] END ..............n_estimators=10;, score=0.736 total time=   3.1s
[CV 3/5; 2/5] START n_estimators=10.............................................
[CV 3/5; 2/5] END ..............n_estimators=10;, score=0.744 total time=   3.1s
[CV 5/5; 2/5] START n_estimators=10.............................................
[CV 5/5; 2/5] END ..............n_estimators=10;, score=0.735 total time=   3.1s
[CV 2/5; 3/5] START n_estimators=25.............................................
[CV 2/5; 3/5] END ..............n_estimators=25

/Users/timofeyzhdanovich/Desktop/Ph@DS/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 1/5; 1/5] START n_estimators=5..............................................
[CV 1/5; 1/5] END ...............n_estimators=5;, score=0.693 total time=   1.6s
[CV 3/5; 1/5] START n_estimators=5..............................................
[CV 3/5; 1/5] END ...............n_estimators=5;, score=0.693 total time=   1.6s
[CV 5/5; 1/5] START n_estimators=5..............................................
[CV 5/5; 1/5] END ...............n_estimators=5;, score=0.689 total time=   1.6s
[CV 2/5; 2/5] START n_estimators=10.............................................
[CV 2/5; 2/5] END ..............n_estimators=10;, score=0.734 total time=   3.1s
[CV 4/5; 2/5] START n_estimators=10.............................................
[CV 4/5; 2/5] END ..............n_estimators=10;, score=0.730 total time=   3.1s
[CV 1/5; 3/5] START n_estimators=25.............................................
[CV 1/5; 3/5] END ..............n_estimators=25;, score=0.817 total time=   7.8s
[CV 3/5; 3/5] START n_estima

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_iter=30,
                   n_jobs=2,
                   param_distributions={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25]),
                                        'max_depth': [3, 4, 5, 6, None],
                                        'min_samples_leaf': array([1, 2, 3, 4, 5]),
                                        'n_estimators': [5, 10, 25, 50, 75]},
                   random_state=42, verbose=10)

Подобранные гиперпараметры:

In [12]:
rnd_boosting_gridsearch.best_params_

{'n_estimators': 50,
 'min_samples_leaf': 1,
 'max_depth': 6,
 'learning_rate': 0.25}

Качество оптимальной модели:

In [13]:
print('train accuracy {:.4f}'.format(
    accuracy_score(rnd_boosting_gridsearch.predict(X_train), y_train)
))
print('test accuracy {:.4f}'.format(
    accuracy_score(rnd_boosting_gridsearch.predict(X_test), y_test)
))

train accuracy 0.9999
test accuracy 0.9498


### 2.3 Подбор значения параметра `learning_rate`

Как вы уже знаете, скорость сходимости, да и сам факт сходимости метода оптимизации сильно зависят от гиперпараметров метода. Одним из наиболее важных гиперпараметров методов оптимизации является `learning_rate`. Поэтому бывает полезно подобрать значение `learning_rate` наиболее точно.

Предварительно зафиксируем остальные найденные параметры.

In [14]:
boosting_gridsearch = GridSearchCV(
    estimator=GradientBoostingClassifier(
        n_estimators=75, max_depth=6, min_samples_leaf=5
    ),
    param_grid={
        'learning_rate': np.linspace(0.05, 0.25, 5),
    },
    cv=5,  # разбиение выборки на 5 фолдов
    verbose=10,  # насколько часто печатать сообщения
    n_jobs=2  # кол-во параллельных процессов
)
boosting_gridsearch.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 3/5; 18/30] START learning_rate=0.15000000000000002, max_depth=None, min_samples_leaf=4, n_estimators=50
[CV 3/5; 18/30] END learning_rate=0.15000000000000002, max_depth=None, min_samples_leaf=4, n_estimators=50;, score=0.946 total time= 1.7min
[CV 5/5; 18/30] START learning_rate=0.15000000000000002, max_depth=None, min_samples_leaf=4, n_estimators=50
[CV 5/5; 18/30] END learning_rate=0.15000000000000002, max_depth=None, min_samples_leaf=4, n_estimators=50;, score=0.940 total time= 1.7min
[CV 5/5; 22/30] START learning_rate=0.1, max_depth=3, min_samples_leaf=5, n_estimators=50
[CV 5/5; 22/30] END learning_rate=0.1, max_depth=3, min_samples_leaf=5, n_estimators=50;, score=0.875 total time=  16.6s
[CV 2/5; 23/30] START learning_rate=0.05, max_depth=4, min_samples_leaf=5, n_estimators=75
[CV 2/5; 23/30] END learning_rate=0.05, max_depth=4, min_samples_leaf=5, n_estimators=75;, score=0.890 total time=  31.3s
[CV 4/5; 23/30] ST

/Users/timofeyzhdanovich/Desktop/Ph@DS/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(max_depth=6,
                                                  min_samples_leaf=5,
                                                  n_estimators=75),
             n_jobs=2,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25])},
             verbose=10)

Подобранное значение `learning_rate`:

In [15]:
boosting_gridsearch.best_params_

{'learning_rate': 0.2}

Качество оптимальной модели:

In [16]:
print('train accuracy {:.4f}'.format(
    accuracy_score(boosting_gridsearch.predict(X_train), y_train)
))
print('test accuracy {:.4f}'.format(
    accuracy_score(boosting_gridsearch.predict(X_test), y_test)
))

train accuracy 1.0000
test accuracy 0.9618
[CV 1/5; 4/5] START learning_rate=0.2...........................................
[CV 1/5; 4/5] END ............learning_rate=0.2;, score=0.951 total time=  42.2s
[CV 3/5; 4/5] START learning_rate=0.2...........................................
[CV 3/5; 4/5] END ............learning_rate=0.2;, score=0.956 total time=  42.2s
[CV 5/5; 4/5] START learning_rate=0.2...........................................
[CV 5/5; 4/5] END ............learning_rate=0.2;, score=0.948 total time=  42.5s
[CV 2/5; 5/5] START learning_rate=0.25..........................................
[CV 2/5; 5/5] END ...........learning_rate=0.25;, score=0.770 total time=  42.4s
[CV 4/5; 5/5] START learning_rate=0.25..........................................
[CV 4/5; 5/5] END ...........learning_rate=0.25;, score=0.952 total time=  42.6s
[CV 4/5; 18/30] START learning_rate=0.15000000000000002, max_depth=None, min_samples_leaf=4, n_estimators=50
[CV 4/5; 18/30] END learning_rate=0.15

Некоторое отличие результата от предыдущего случая можно объяснить случайностью, которая есть в модели. Ее можно зафиксировать, установив параметр `random_state`.